In [37]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rottentomatoes-400k-review/rottentomatoes-400k.csv
/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/credits.csv
/kaggle/input/the-movies-dataset/keywords.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/ratings_small.csv
/kaggle/input/the-movies-dataset/links.csv
/kaggle/input/rotten-tomatoes-movie-database/all_movie.csv


In [38]:
import pandas as pd
import json
import numpy as np

df_movie_info_raw = pd.read_csv("/kaggle/input/the-movies-dataset/movies_metadata.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [44]:
df_movie_data = df_movie_info_raw[["title", "overview", "budget", "revenue"]].replace(0, np.nan)
df_movie_data["budget"].replace('/ff9qCepilowshEtG2GYWwzt2bs4.jpg', 0, inplace=True)
df_movie_data["budget"].replace('/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg', 0, inplace=True)
df_movie_data["budget"].replace('/zaSf5OG7V8X8gqFvly88zDdRm46.jpg', 0, inplace=True)
df_movie_data["budget"] = df_movie_data["budget"].astype(float).replace(0, np.nan)

def get_genres_in_one_str(x): #x is json obj in str form
    x = x.strip('][').replace("},", "}},").split('}, ')
    if x == [""]:
        return np.nan
    genres = []
    for obj in x:
        obj = obj.replace("\'", "\"")
        genre = json.loads(obj)["name"]
        genres.append(genre)
    return "|".join(sorted(genres))

df_movie_data["all genres"] = df_movie_info_raw["genres"].apply(get_genres_in_one_str)
df_movie_data.dropna(inplace=True)
df_movie_data.rename(columns={"revenue": "box office"}, inplace=True)
df_movie_data.drop_duplicates(subset=['title'], inplace=True)
df_movie_data.set_index("title", inplace=True)
df_movie_data = df_movie_data[(df_movie_data["budget"] >= df_movie_reviews["budget"].quantile(0.05))]
#                              & (df_movie_data["box office"] >= df_movie_reviews["box office"].quantile(0.25))]
df_movie_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 160 entries, Waterworld to Transformers: The Last Knight
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   overview    160 non-null    object 
 1   budget      160 non-null    float64
 2   box office  160 non-null    float64
 3   all genres  160 non-null    object 
dtypes: float64(2), object(2)
memory usage: 6.2+ KB


In [51]:
df_movies2 = pd.read_csv("/kaggle/input/rotten-tomatoes-movie-database/all_movie.csv")[["Title", "Description", "Rating"]]
df_movies2.dropna(inplace=True)
df_movies2.rename(columns={"Rating": "Film Rating"}, inplace=True)
df_movies2.drop_duplicates(subset=['Title'], inplace=True)
df_movies2.set_index("Title", inplace=True)

#combine the two movie database datasets
df_movie_info = df_movies2.merge(df_movie_data, left_index=True, right_index=True, how="inner")
df_movie_info.drop("Description", axis=1, inplace=True)
df_movie_info.rename(columns={"overview": "Description"}, inplace=True)
df_movie_info.index = df_movie_info.index.map(lambda x: x.upper())
df_movie_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 142 entries, CAPTAIN AMERICA: CIVIL WAR to TOMORROWLAND
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Film Rating  142 non-null    object 
 1   Description  142 non-null    object 
 2   budget       142 non-null    float64
 3   box office   142 non-null    float64
 4   all genres   142 non-null    object 
dtypes: float64(2), object(3)
memory usage: 6.7+ KB


In [53]:
df_movie_reviews = pd.read_csv("/kaggle/input/rottentomatoes-400k-review/rottentomatoes-400k.csv")
df_movie_reviews = df_movie_reviews[["Movie", "Review", "Score"]]
df_movie_reviews.set_index("Movie", inplace=True)
df_movie_reviews["Average Movie Score"] = df_movie_reviews.groupby("Movie")["Score"].mean()


#final dataset below; combines everything from the 3 datasets used in this notebook
df_movie_reviews = df_movie_reviews.merge(df_movie_info, left_index=True, right_index=True, how="inner")
df_movie_reviews.reset_index(inplace=True)
df_movie_reviews.rename(columns={"index": "Movie"}, inplace=True)
#get all genres into seperate columns
df_genres = df_movie_reviews["all genres"].str.get_dummies(sep='|')
df_movie_reviews = df_movie_reviews.merge(df_genres, left_index=True, right_index=True, how="inner")
df_movie_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18713 entries, 0 to 18712
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Movie                18713 non-null  object 
 1   Review               18713 non-null  object 
 2   Score                18713 non-null  float64
 3   Average Movie Score  18713 non-null  float64
 4   Film Rating          18713 non-null  object 
 5   Description          18713 non-null  object 
 6   budget               18713 non-null  float64
 7   box office           18713 non-null  float64
 8   all genres           18713 non-null  object 
 9   Action               18713 non-null  int64  
 10  Adventure            18713 non-null  int64  
 11  Animation            18713 non-null  int64  
 12  Comedy               18713 non-null  int64  
 13  Crime                18713 non-null  int64  
 14  Drama                18713 non-null  int64  
 15  Family               18713 non-null 

In [54]:
#write to csv
df_movie_reviews.to_csv("movie_reviews_with_info.csv")

df_movie_reviews.head()

,Movie,Review,Score,Average Movie Score,Film Rating,Description,budget,box office,all genres,Action,...,Crime,Drama,Family,Fantasy,Mystery,Romance,Science Fiction,Thriller,War,Western
0,47 RONIN,"Heavy on CGI, light on entertainment, 47 Ronin...",40.0,44.5,PG-13,"Based on the original 1941 movie from Japan, a...",175000000.0,150962475.0,Action|Adventure|Drama|Fantasy,1,...,0,1,0,1,0,0,0,0,0,0
1,47 RONIN,"Opening on Christmas Day, 47 Ronin obviously i...",37.5,44.5,PG-13,"Based on the original 1941 movie from Japan, a...",175000000.0,150962475.0,Action|Adventure|Drama|Fantasy,1,...,0,1,0,1,0,0,0,0,0,0
2,47 RONIN,The whole debacle cost more than 200 million b...,30.0,44.5,PG-13,"Based on the original 1941 movie from Japan, a...",175000000.0,150962475.0,Action|Adventure|Drama|Fantasy,1,...,0,1,0,1,0,0,0,0,0,0
3,47 RONIN,Feels like it was yanked out of some bargain b...,20.0,44.5,PG-13,"Based on the original 1941 movie from Japan, a...",175000000.0,150962475.0,Action|Adventure|Drama|Fantasy,1,...,0,1,0,1,0,0,0,0,0,0
4,47 RONIN,"At its best, 47 Ronin is an elegant paean to a...",79.0,44.5,PG-13,"Based on the original 1941 movie from Japan, a...",175000000.0,150962475.0,Action|Adventure|Drama|Fantasy,1,...,0,1,0,1,0,0,0,0,0,0
